In [2]:
# tensorflow.keras의 데이터셋 중 reuters 데이터셋을 사용해서 분류 모델 개발
# 1. 데이터 가져오기
# 2. 데이터의 특성 파악
# 3. 입력 데이터 구성
# 4. 모델 개발
# 5. 모델 평가

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras as tf_keras

In [4]:
# 데이터 준비
(X_train, y_train), (X_test, y_test) = tf_keras.datasets.reuters.load_data(num_words=10000) # 10000 개의 단어 집합 사용

2110848/2110848 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
# 단어 사전 확인
word_to_index = tf_keras.datasets.reuters.get_word_index()

550378/550378 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
# 단어 번호로 단어를 찾기 위한 맵 만들기
index_to_word = { value: key for key, value in word_to_index.items() }

In [7]:
# 각 문장을 BOW 형식으로 변환 -> (1, 10000)
def vectorize_sentences(sentences, dimension=10000, bow=True): # dimension : column, 전체단어갯수, sentences : 행, 문장들
    results = np.zeros((len(sentences), dimension))

    for i, sentence in enumerate(sentences):
        for word in sentence:
            if bow:
              results[i, word] += 1.  # 단어 위치에 발생 빈도 encoding
            else:
              results[i, word] = 1.   # 단어 위치에 1 encoding

    return results

In [8]:
# 입력 데이터 변환
X_train2 = vectorize_sentences(X_train, bow=False)
X_test2 = vectorize_sentences(X_test, bow=False)

X_train3 = vectorize_sentences(X_train, bow=True)
X_test3 = vectorize_sentences(X_test, bow=True)

In [9]:
# target 데이터 특성 확인
print(y_train.shape)
np.unique(y_train, return_counts=True)

(8982,)


(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]),
 array([  55,  432,   74, 3159, 1949,   17,   48,   16,  139,  101,  124,
         390,   49,  172,   26,   20,  444,   39,   66,  549,  269,  100,
          15,   41,   62,   92,   24,   15,   48,   19,   45,   39,   32,
          11,   50,   10,   49,   19,   19,   24,   36,   30,   13,   21,
          12,   18]))

In [10]:
# 모델 구조 설계 1

base_model = tf_keras.models.Sequential()
base_model.add(tf_keras.layers.Input(shape=(10000,)))
base_model.add(tf_keras.layers.Dense(512, activation='relu'))
base_model.add(tf_keras.layers.Dense(256, activation='relu'))
base_model.add(tf_keras.layers.Dense(128, activation='relu'))
base_model.add(tf_keras.layers.Dense(46, activation='softmax')) # 다중분류인 경우 activation=softmax, 유닛갯수는 분류 갯수 사용

base_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 512)                 │       5,120,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 46)                  │           5,934 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,290,670 (20.18 MB)

 Trainable params: 5,290,670 (20.18 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# 모델 구조 설계 2 : 가중치 초기화 설정

model1 = tf_keras.models.Sequential()
model1.add(tf_keras.layers.Input(shape=(10000,)))
model1.add(tf_keras.layers.Dense(512, activation='relu', kernel_initializer="he_uniform"))
model1.add(tf_keras.layers.Dense(256, activation='relu', kernel_initializer="he_uniform"))
model1.add(tf_keras.layers.Dense(128, activation='relu', kernel_initializer=tf_keras.initializers.HeUniform(seed=42)))
model1.add(tf_keras.layers.Dense(46, activation='softmax')) # 다중분류인 경우 activation=softmax, 유닛갯수는 분류 갯수 사용

model1.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 512)                 │       5,120,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 46)                  │           5,934 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,290,670 (20.18 MB)

 Trainable params: 5,290,670 (20.18 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# 모델 구조 설계 3 : 배치 정규화

model2 = tf_keras.models.Sequential()
model2.add(tf_keras.layers.Input(shape=(10000,)))
model2.add(tf_keras.layers.Dense(512))
model2.add(tf_keras.layers.BatchNormalization())
model2.add(tf_keras.layers.Activation('relu'))

model2.add(tf_keras.layers.Dense(256))
model2.add(tf_keras.layers.BatchNormalization())
model2.add(tf_keras.layers.Activation('relu'))

model2.add(tf_keras.layers.Dense(128))
model2.add(tf_keras.layers.BatchNormalization())
model2.add(tf_keras.layers.Activation('relu'))

model2.add(tf_keras.layers.Dense(46, activation='softmax')) # 다중분류인 경우 activation=softmax, 유닛갯수는 분류 갯수 사용

model2.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                      │ (None, 512)                 │       5,120,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_2 (Activation)            │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 46)                  │           5,934 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,294,254 (20.20 MB)

 Trainable params: 5,292,462 (20.19 MB)

 Non-trainable params: 1,792 (7.00 KB)

In [31]:
# 모델 구조 설계 4 : 기중치 규제 (L1.)

model3 = tf_keras.models.Sequential()
model3.add(tf_keras.layers.Input(shape=(10000,)))
model3.add(tf_keras.layers.Dense(512, activation='relu', kernel_regularizer=tf_keras.regularizers.L2(0.001)))
model3.add(tf_keras.layers.Dense(256, activation='relu', kernel_regularizer=tf_keras.regularizers.L2(0.001)))
model3.add(tf_keras.layers.Dense(128, activation='relu', kernel_regularizer=tf_keras.regularizers.L2(0.001)))
model3.add(tf_keras.layers.Dense(46, activation='softmax')) # 다중분류인 경우 activation=softmax, 유닛갯수는 분류 갯수 사용

model3.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_38 (Dense)                     │ (None, 512)                 │       5,120,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_39 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_40 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_41 (Dense)                     │ (None, 46)                  │           5,934 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,290,670 (20.18 MB)

 Trainable params: 5,290,670 (20.18 MB)

 Non-trainable params: 0 (0.00 B)

In [32]:
# 모델 구조 설계 5 : 드롭아웃 적용

model4 = tf_keras.models.Sequential()
model4.add(tf_keras.layers.Input(shape=(10000,)))
model4.add(tf_keras.layers.Dense(512, activation='relu'))
model4.add(tf_keras.layers.Dropout(0.5, seed=42))
model4.add(tf_keras.layers.Dense(256, activation='relu'))
model4.add(tf_keras.layers.Dropout(0.5, seed=42))
model4.add(tf_keras.layers.Dense(128, activation='relu'))
model4.add(tf_keras.layers.Dropout(0.5, seed=42))
model4.add(tf_keras.layers.Dense(46, activation='softmax')) # 다중분류인 경우 activation=softmax, 유닛갯수는 분류 갯수 사용

model4.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_42 (Dense)                     │ (None, 512)                 │       5,120,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_43 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_44 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_45 (Dense)                     │ (None, 46)                  │           5,934 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,290,670 (20.18 MB)

 Trainable params: 5,290,670 (20.18 MB)

 Non-trainable params: 0 (0.00 B)

In [33]:
# 모델 훈련 설계

base_model.compile(loss='sparse_categorical_crossentropy',
                   optimizer='adam',
                   metrics=['accuracy'])
model1.compile(loss='sparse_categorical_crossentropy',
               optimizer='adam',
               metrics=['accuracy'])
model2.compile(loss='sparse_categorical_crossentropy',
               optimizer='adam',
               metrics=['accuracy'])
model3.compile(loss='sparse_categorical_crossentropy',
               optimizer='adam',
               metrics=['accuracy'])
model4.compile(loss='sparse_categorical_crossentropy',
               optimizer='adam',
               metrics=['accuracy'])

In [34]:
# fit_history_base = base_model.fit(X_train2, y_train, epochs=10, batch_size=256, validation_split=0.2)
# fit_history_1 = model1.fit(X_train2, y_train, epochs=10, batch_size=256, validation_split=0.2)
# fit_history_2 = model2.fit(X_train2, y_train, epochs=10, batch_size=256, validation_split=0.2)
# fit_history_3 = model3.fit(X_train2, y_train, epochs=10, batch_size=256, validation_split=0.2)
fit_history_4 = model4.fit(X_train2, y_train, epochs=10, batch_size=256, validation_split=0.2)

Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 7s 185ms/step - accuracy: 0.2505 - loss: 3.2110 - val_accuracy: 0.5804 - val_loss: 1.6960
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 9s 162ms/step - accuracy: 0.5943 - loss: 1.6970 - val_accuracy: 0.6978 - val_loss: 1.3920
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 7s 230ms/step - accuracy: 0.6646 - loss: 1.4175 - val_accuracy: 0.7201 - val_loss: 1.2276
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 197ms/step - accuracy: 0.7197 - loss: 1.1649 - val_accuracy: 0.7429 - val_loss: 1.1424
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 12s 250ms/step - accuracy: 0.7595 - loss: 0.9594 - val_accuracy: 0.7613 - val_loss: 1.1101
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 8s 167ms/step - accuracy: 0.7895 - loss: 0.8324 - val_accuracy: 0.7735 - val_loss: 1.0779
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 7s 243ms/step - accuracy: 0.8179 - loss: 0.7165 - val_accuracy: 0.7824 - val_loss: 1.0653
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 173ms/step - accuracy: 0.8461 - loss: 0.6253 - val_accuracy: 0

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [35]:
print( base_model.evaluate(X_train2, y_train), base_model.evaluate(X_test2, y_test) )
print( model1.evaluate(X_train2, y_train), model1.evaluate(X_test2, y_test) )
print( model2.evaluate(X_train2, y_train), model2.evaluate(X_test2, y_test) )
print( model3.evaluate(X_train2, y_train), model2.evaluate(X_test3, y_test) )
print( model4.evaluate(X_train2, y_train), model2.evaluate(X_test3, y_test) )

281/281 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.0138 - loss: 3.8223
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.0082 - loss: 3.8251
[3.8218016624450684, 0.013137386180460453] [3.8228657245635986, 0.010685663670301437]
281/281 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.0036 - loss: 3.8823
71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.0047 - loss: 3.8790
[3.8809142112731934, 0.004342017229646444] [3.878870964050293, 0.005788067821413279]
281/281 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9697 - loss: 0.0688
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7984 - loss: 1.2719
[0.2949465811252594, 0.9404364228248596] [1.323474645614624, 0.7965271472930908]
281/281 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step - accuracy: 0.0105 - loss: 5.3271
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7932 - loss: 1.1891
[5.3266448974609375, 0.01146737951785326] [1.2746682167053223, 0.780053436756134]
281/281 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9295 - los